In [6]:
import pandas as pd
from efficient_apriori import apriori



In [7]:
base = pd.read_csv("bases/bases_limpas/base_para_modelo_final.csv")

In [8]:
base.columns.tolist()

['Numero nota fiscal',
 'Data da venda',
 'Código produto',
 'Descrição do produto',
 'Quantidade do produto',
 'Valor unitário',
 'Preço de custo',
 'Valor total produto',
 'Valor da nota',
 'Código da categoria',
 'Categoria',
 'Código da Marca',
 'Marca',
 'Quantidade estoque',
 'Margem bruta',
 'Margem %',
 'Markup']

In [9]:
def recomendar_substitutos(df, cod_produto_pesquisado, n_recomendacoes=5):
    # 1. Localiza o produto base
    produto_base = df[df['Código produto'] == cod_produto_pesquisado]

    if produto_base.empty:
        return "Produto não encontrado"

    produto_base = produto_base.iloc[0]

    # 2. Verifica se o produto está sem estoque
    if produto_base['Quantidade estoque'] > 0:
        return "Produto em estoque, recomendação não necessária"

    # 3. Filtra produtos da mesma categoria, com estoque e código diferente
    candidatos = df[
        (df['Código da categoria'] == produto_base['Código da categoria']) &
        (df['Código produto'] != cod_produto_pesquisado) &
        (df['Quantidade estoque'] > 0)
    ].copy()

    if candidatos.empty:
        return "Sem produtos semelhantes em estoque"

    # 4. Remove duplicatas por código de produto
    candidatos = candidatos.drop_duplicates(subset='Código produto')

    # 5. Calcula distância de preço e margem %
    candidatos['dist_preco'] = (candidatos['Valor unitário'] - produto_base['Valor unitário']).abs()
    candidatos['dist_margem'] = (candidatos['Margem %'] - produto_base['Margem %']).abs()

    # 6. Ordena pelos mais similares
    recomendados = candidatos.sort_values(
        by=['dist_preco', 'dist_margem']
    ).head(n_recomendacoes)

    return recomendados[['Código produto', 'Descrição do produto', 'Valor unitário', 'Margem %', 'Marca', 'Quantidade estoque']]


In [10]:
recomendar_substitutos(base, 35494)

,Código produto,Descrição do produto,Valor unitário,Margem %,Marca,Quantidade estoque
30520,35485,TNT TECIDO 50M 37G AZ CE DECOFIX,54.14,0.14,JC DECOR PLASTICOS E TECIDOS LTDA,4
2102,21422,TNT TECIDO 50M 35G AZ CE NEWTNT,54.37,0.12,NEWTNT INDUSTRIA DE TECIDOS LTDA,15
2886,29097,TNT TECIDO 50M 35G PK NEWTNT,54.78,0.12,NEWTNT INDUSTRIA DE TECIDOS LTDA,33
2141,35487,TNT TECIDO 50M 37G AZ RY DECOFIX,53.40,0.13,JC DECOR PLASTICOS E TECIDOS LTDA,32
94065,14797,TNT TECIDO 50M RS SUL BRASIL,55.81,0.14,SUL BRASIL IND E COM DE ACESSORIOS DE PLAST E ...,2
